In [1]:
#IMPORTANT NOTE! RUN THIS CODE ONLY WHEN YOU DO NOT PLAN TO USE YOUR COMPUTER! THIS COULD RUN FOR MULTIPLE DAYS!
import kerastuner as kt
import tensorflow as tf
import pandas as pd
from datetime import datetime
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split
import IPython
import random
import matplotlib.pyplot as plt
import copy
import h5py    
import pickle

%load_ext tensorboard


In [2]:
#IMPORTANT NOTE! RUN THIS CODE ONLY WHEN YOU DO NOT PLAN TO USE YOUR COMPUTER! THIS COULD RUN FOR MULTIPLE DAYS!
data = open("data.pkl",'rb')
data = pickle.load(data,encoding='bytes')




lijst = copy.copy(data)
voc_List = lijst.pop("Bron")


for naam in voc_List:
  vocabulair = voc_List.unique()



Y= data.pop("Bron")


vertaling = pd.DataFrame({
    "0" :[],
    "1" :[]
})



for i in range(0,len(Y)):
  a = np.where(vocabulair == Y[i])
  a = int(a[0])
  if(a == 1):
    Cs = 1
    Bg = 0
  else:
    Cs = 0
    Bg = 1
  vertaling = vertaling.append({
      "0": Bg,
      "1": Cs
  }, ignore_index = True)


vertaling = np.array(vertaling)
Peak_Y = data.pop('Peak')
Peak_Y = np.array(Peak_Y)
Peak_Y = Peak_Y.tolist()



Z = data.pop("Counts")
X= data.pop("Energie")

Y2 = data.pop("Activiteit")
Y2 = Y2.tolist()
for i in range (0, len(Y2)):
  a = Y2[i]
  a = a/1000
  Y2[i]= a


Y2 = np.array(Y2)

spectrum = pd.DataFrame()
energie = []
counts = []
n =random.randint(0,len(X))

for i in range(0, len(X)):
  a = X[i]
  b = Z[i]
  a = np.array(a)

  b = np.array(b)

  energie.append(a)
  counts.append(b)


  
spectrum['Counts'] = counts
spectrum['Energie']= energie
spectrum = spectrum.to_numpy()
spectrum = spectrum.tolist()

counts = np.array(counts)
energie = np.array(energie)
x_train, x_test, y_train, y_test = train_test_split(spectrum,vertaling, test_size=0.1, random_state = 42)



x_test_spect = x_test.copy()


x_train = tf.convert_to_tensor(x_train)
y_train = tf.convert_to_tensor(y_train)
x_test = tf.convert_to_tensor(x_test)
y_test = tf.convert_to_tensor(y_test)

x2_train,x2_test,y2_train,y2_test = train_test_split(data,Peak_Y, test_size = 0.1, random_state=42)

y2_train = tf.convert_to_tensor(y2_train)
y2_test = tf.convert_to_tensor(y2_test)

In [3]:
#IMPORTANT NOTE! RUN THIS CODE ONLY WHEN YOU DO NOT PLAN TO USE YOUR COMPUTER! THIS COULD RUN FOR MULTIPLE DAYS!
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Concatenate, Conv1D
from datetime import datetime
now = datetime.now()


def model_builder1(hp):
    
    
    hp_units1 = hp.Int('units1', min_value=2, max_value=512, step=2, default = 128)
    hp_units10 = hp.Int('units10', min_value=2, max_value=128, step=2, default = 64)
    hp_units11 = hp.Int('units11', min_value=2, max_value=128, step=2, default = 64)
    hp_units12 = hp.Int('units12', min_value=2, max_value=128, step=2, default = 64)
    hp_units13 = hp.Int('units13', min_value=2, max_value=128, step=2, default = 64)
    hp_units14 = hp.Int('units14', min_value=2, max_value=128, step=2, default = 64)
    hp_units15 = hp.Int('units15', min_value=2, max_value=128, step=2, default = 64)
    hp_units16 = hp.Int('units16', min_value=2, max_value=128, step=2, default = 64)
    
    input1 = keras.Input(shape =(2,4096), name = "Counts")  
    
    #Implementatie Faster R-CNN 1st CONV Layers om feature maps op te stellen
    x1 = tf.keras.layers.Convolution1D(hp_units1,1, strides = 1,padding = 'same', activation = 'swish', name = 'block1_conv1')(input1)
    x1 = tf.keras.layers.Convolution1D(hp_units1,1, strides = 1, padding = 'same', activation= 'swish', name = 'block1_conv2')(x1)
    x1 = tf.keras.layers.Convolution1D(hp_units1,1, strides = 1,padding = 'same', activation = 'swish', name = 'block1_conv3')(x1) 
    x1 = tf.keras.layers.MaxPooling1D(pool_size = 1, strides = 1, padding = 'same', name = 'Pooling1')(x1)

    x2 = tf.keras.layers.concatenate([x1,input1])
    x2 = tf.keras.layers.Convolution1D(hp_units1,1 , padding = 'same', activation = 'swish', name = 'block2_conv1')(x2)
    x2 = tf.keras.layers.Convolution1D(hp_units1,1 ,padding = 'same', activation = 'swish')(x2)
    x2 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x2)
    x2 = tf.keras.layers.MaxPooling1D(pool_size = 1, strides = 1, padding = 'same', name = 'Pooling4')(x2)
    
    x3 = tf.keras.layers.concatenate([input1,x1,x2])
    x3 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x3)
    x3 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x3)
    x3 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x3)
    x3 = tf.keras.layers.MaxPooling1D(pool_size = 1, strides = 1, padding = 'same', name = 'Pooling5')(x3)
    
    x4 = tf.keras.layers.concatenate([input1,x1,x2,x3])
    x4 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x4)
    x4 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x4)
    x4 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x4)
    x4 = tf.keras.layers.MaxPooling1D(pool_size = 1, strides = 1, padding = 'same', name = 'Pooling6')(x4)
    
    x5 = tf.keras.layers.concatenate([input1,x1,x2,x3,x4])
    x5 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x5)
    x5 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x5)
    x5 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x5) 
    x5 = tf.keras.layers.MaxPooling1D(pool_size = 1, strides = 1, padding = 'same', name = 'Pooling7')(x5)
     
    x6 = tf.keras.layers.concatenate([input1,x1,x2,x3,x4,x5])
    x6 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x6)
    x6 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x6)
    x6 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x6)
    x6 = tf.keras.layers.MaxPooling1D(pool_size = 1, strides = 1, padding = 'same', name = 'Pooling8')(x6)
    
    x7 = tf.keras.layers.concatenate([input1,x1,x2,x3,x4,x6,x5])
    x7 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x7)
    x7 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x7)
    x7 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x7) 
    x7 = tf.keras.layers.MaxPooling1D(pool_size = 1, strides = 1, padding = 'same', name = 'Pooling9')(x7)
    
    x8= tf.keras.layers.concatenate([input1,x1,x2,x3,x4,x6,x7,x5])
    x8 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x8)
    x8 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x8)
    x8 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x8) 
    x8 = tf.keras.layers.MaxPooling1D(pool_size = 1, strides = 1, padding = 'same', name = 'Pooling10')(x8)
    
    x9= tf.keras.layers.concatenate([input1,x1,x2,x3,x4,x6,x7,x5,x8])
    x9 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish')(x9)
    x9 = tf.keras.layers.Convolution1D( hp_units1,1,padding = 'same', activation = 'swish')(x9)
    x9 = tf.keras.layers.Convolution1D(hp_units1,1,padding = 'same', activation = 'swish', name = 'block3_conv3')(x9) 
    x9 = tf.keras.layers.MaxPooling1D(pool_size = 1, strides = 1, name = 'Pooling11')(x9)
    
    conc = tf.keras.layers.concatenate([input1,x1,x2,x3,x4,x5,x6,x7,x8,x9])
    conc = tf.keras.layers.MaxPooling1D(pool_size = 1, strides = 8, name = 'MaxPool')(conc)
    x = tf.keras.layers.Flatten()(conc) 
    output2 = tf.keras.layers.Dense(4096, activation = 'sigmoid', name = 'Peak')(x)
    x = tf.keras.layers.Dense(units = hp_units10, activation = 'relu', name = 'Dense1')(x)
    x = tf.keras.layers.Dense(units = hp_units11, activation = 'relu', name = 'Dense2')(x)
    x = tf.keras.layers.Dense(units = hp_units12, activation = 'relu', name = 'Dense3')(x)
    x = tf.keras.layers.Dense(units = hp_units13, activation = 'relu', name = 'Dense4')(x)
    x = tf.keras.layers.Dense(units = hp_units14, activation = 'relu', name = 'Dense5')(x)
    x = tf.keras.layers.Dense(units = hp_units15, activation = 'relu', name = 'Dense6')(x)
    x = tf.keras.layers.Dense(units = hp_units16, activation = 'relu', name = 'Dense7')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    output1 = tf.keras.layers.Dense(2, activation = 'softmax', name = 'Bron')(x)
   

    model = keras.Model(inputs = (input1), outputs=(output1, output2))
    
    
    
    losses = {
        'Bron' : tf.keras.losses.CategoricalCrossentropy(from_logits = True),
        'Peak' : tf.keras.losses.BinaryCrossentropy()
    }
    
    
    # precision = true positive / ( true positive + false negative), recall = true positive/ (true positive + false positive)
    Metrics = {
        'Bron' : [tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), 'accuracy'],
        'Peak': [tf.keras.metrics.BinaryAccuracy()]
        
    }
    
    
    
    model.compile(loss=losses,
                  optimizer=keras.optimizers.Adagrad(),
                  metrics= Metrics )
    
  
    return model


In [4]:
#IMPORTANT NOTE! RUN THIS CODE ONLY WHEN YOU DO NOT PLAN TO USE YOUR COMPUTER! THIS COULD RUN FOR MULTIPLE DAYS!
!rm -rf ./logs/ 

tuner = kt.tuners.bayesian.BayesianOptimization(model_builder1,
                                                kt.Objective(("val_Bron_recall"), direction="max"),
                                                50)

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
#IMPORTANT NOTE! RUN THIS CODE ONLY WHEN YOU DO NOT PLAN TO USE YOUR COMPUTER! THIS COULD RUN FOR MULTIPLE DAYS!
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = './logs/')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_Bron_recall', patience=5)

In [ ]:
#IMPORTANT NOTE! RUN THIS CODE ONLY WHEN YOU DO NOT PLAN TO USE YOUR COMPUTER! THIS COULD RUN FOR MULTIPLE DAYS!
tuner.search(x_train, (y_train,y2_train),batch_size = 16, epochs=25, verbose = 1, validation_data = (x_test,(y_test,y2_test)), callbacks=[stop_early])

Trial 34 Complete [00h 13m 44s]
val_Bron_recall: 0.881538450717926

Best val_Bron_recall So Far: 0.9615384340286255
Total elapsed time: 20h 21m 32s

Search: Running Trial #35

Hyperparameter    |Value             |Best Value So Far 
units1            |478               |462               
units10           |116               |112               
units11           |84                |72                
units12           |56                |64                
units13           |116               |84                
units14           |44                |50                
units15           |80                |86                
units16           |42                |42                

Epoch 1/25
366/366 [==============================] - ETA: 0s - loss: 1.3458 - Bron_loss: 0.6416 - Peak_loss: 0.7041 - Bron_recall: 0.5711 - Bron_precision: 0.5711 - Bron_accuracy: 0.5711 - Peak_binary_accuracy: 0.5242

In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, (y_train, y2_train), epochs=50, validation_data = (x_test,(y_test,y2_test)))

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))


hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(x_train, (y_train, y2_train), epochs=best_epoch, validation_data = (x_test,(y_test,y2_test)))

In [ ]:
hypermodel.fit()